In [1]:
%reload_ext autoreload
%autoreload 2

## CellLogger subsystem

This test tests the CellLogger subsystem, accessible via `exp.cl` once one of the `IPyExperiments` subclasses has been initiated.

## Test specifics

Since we need to validate the the output, we have to capture it first. The way jupyter is setup, is that in once cell you set up a capture with `%%capture` magick and then in the next cell you can analyze it. That's why each test group has two cells, the first one doing the action to be tested and the following one doing the validatations.

Moreover, the output of this test becomes confusing because the capture mechanism somehow messes things up which leads to re-running the `post_run_cell` callback of the CellLogger subsystem again - as a result you get a bogus output with 0's regardless of the code being run. It doesn't interfere with the testing, but it does interfere with things like `.data` which gets reset because of that, showing invalid information - therefore we can only test `.data` w/o capturing the cell's output.

## Setup

In [2]:
from math import isclose
from ipyexperiments import *
import re, numpy as np
import torch

In [3]:
def consume_cpu_ram(n): return np.ones((n, n))
def consume_gpu_ram(n): return torch.ones((n, n)).cuda()
def consume_cpu_ram_128mb():  return consume_cpu_ram(2**12)
def consume_gpu_ram_1024mb(): return consume_gpu_ram(2**14)

In [4]:
# --------------------------------------------------------------------- #
# the following functions work with the captured output
# output is captured by `%%capture output` from a cell before

# convert .data outputs to the same dimensions (MBs) as reports
def b2mbi(x): print(x); return int(x/2**20)

# sample:
# RAM: Consumed Peaked  Used Total | Exec time 0.046s
def check_report_strings(output):
    # basic checks
    to_match = [r'Consumed', 'Peaked']
    for s in to_match: assert re.search(s, output), f"expecting string: {s}"

# sample:        
# CPU:      123    321     2159 MB
# GPU:      356    789     2160 MB
def get_sizes(output, type):
    match = re.findall(type + r': +([\d\.]+) +([\d\.]+) +([\d\.]+) MB', output)
    (consumed, peaked, total) = map(float, match[0])
    return consumed, peaked, total

def get_sizes_cpu(output): return get_sizes(output, "CPU")
def get_sizes_gpu(output): return get_sizes(output, "GPU")

# compare reported numbers against expected
def check_match(consumed_reported, peaked_reported, 
                consumed_expected, peaked_expected, abs_tol=0):
    assert isclose(consumed_reported, consumed_expected, abs_tol=abs_tol), f"Consumed RAM reported: {consumed_reported} == real: {consumed_expected}"
    assert isclose(peaked_reported,   peaked_expected,   abs_tol=abs_tol), f"Peaked RAM reported: {peaked_reported} == real: {peaked_expected}"

# these functions extract the reported data from the output
def check_report_cpu(output, consumed_expected, peaked_expected, abs_tol=0):
    consumed_reported, peaked_reported, total_reported = get_sizes_cpu(output)
    check_match(consumed_reported, peaked_reported, 
                consumed_expected, peaked_expected, abs_tol)

def check_report_gpu(output, consumed_expected, peaked_expected, abs_tol=0):
    consumed_reported, peaked_reported, total_reported = get_sizes_gpu(output)
    check_match(consumed_reported, peaked_reported, 
                consumed_expected, peaked_expected, abs_tol)
def print_output(output):
    print("Captured output:\n" + "-"*50 + "\n" + output + "-"*50 + "\n")

## Consume

In [16]:
#if 'il' in locals(): il.stop() # helps debug
exp1 = IPyExperimentsPytorch(enable=False)


*** Experiment started with the Pytorch backend
Device: ID 0, GeForce GTX 1070 Ti (7.9 GB RAM)

RAM: Consumed Peaked  Used Total | Exec time 0.000s
CPU:        0      0     2159 MB |
GPU:        0      0      996 MB |


In [6]:
%%capture output
cpu1 = consume_cpu_ram_128mb()
gpu1 = consume_gpu_ram_1024mb()

RAM: Consumed Peaked  Used Total | Exec time 0.367s
CPU:        0      0     2287 MB |
GPU:        0      0     2020 MB |


In [7]:
"""test_report"""
output = str(output)
print_output(output)

check_report_strings(output)
check_report_cpu(output, consumed_expected= 128, peaked_expected=0, abs_tol=2)
check_report_gpu(output, consumed_expected=1024, peaked_expected=0, abs_tol=0)

# cleanup
del cpu1, gpu1

'test_report'

Captured output:
--------------------------------------------------
RAM: Consumed Peaked  Used Total | Exec time 0.357s
CPU:      128      0     2287 MB |
GPU:     1024      0     2020 MB |
--------------------------------------------------

RAM: Consumed Peaked  Used Total | Exec time 0.010s
CPU:        0      0     2159 MB |
GPU:    -1024   1024      996 MB |


## Consume/release leading to positive peak numbers

In [8]:
%%capture output
# test peak measurement
# here we consume 256MB of RAM and release 128MB 
# testing: Consumed 128, Peaked 128
cpu1 = consume_cpu_ram_128mb()
cpu2 = consume_cpu_ram_128mb()
del cpu1

# here we consume 2048MB of RAM and release 1024MB
# testing: Consumed 1024, Peaked 1024
gpu1 = consume_gpu_ram_1024mb()
gpu2 = consume_gpu_ram_1024mb()
del gpu1


RAM: Consumed Peaked  Used Total | Exec time 0.730s
CPU:        0      0     2287 MB |
GPU:        0   1024     2020 MB |


In [9]:
"""test_peak_memory_usage"""
output = str(output)
print_output(output)

check_report_cpu(output, consumed_expected= 128, peaked_expected= 128, abs_tol=2)
check_report_gpu(output, consumed_expected=1024, peaked_expected=1024, abs_tol=2)

# cleanup
del cpu2, gpu2

'test_peak_memory_usage'

Captured output:
--------------------------------------------------
RAM: Consumed Peaked  Used Total | Exec time 0.719s
CPU:      128    128     2287 MB |
GPU:     1024   1024     2020 MB |
--------------------------------------------------

RAM: Consumed Peaked  Used Total | Exec time 0.005s
CPU:        0      0     2159 MB |
GPU:    -1024   1024      996 MB |


## .data accessor validation

In [10]:
# no capture! breaks .data since it re-runs the post_run_cell, again, resetting .data
# here we consume 256MB of RAM and release 128MB - so that we can test peak measurement
# testing: Consumed 128, Peaked 128
cpu1 = consume_cpu_ram_128mb()
cpu2 = consume_cpu_ram_128mb()
del cpu1
#assert 5==6, "really?"
# here we consume 256MB of RAM and release 128MB - so that we can test peak measurement
# testing: 
gpu1 = consume_gpu_ram_1024mb()
gpu2 = consume_gpu_ram_1024mb()
## Consume/Release Positive Peak
del gpu1


RAM: Consumed Peaked  Used Total | Exec time 0.765s
CPU:      128    128     2287 MB |
GPU:     1024   1024     2020 MB |


In [11]:
"""test_data_accessor"""
cpu_mem, gpu_mem, time_data = exp1.cl.data

check_match(consumed_reported=cpu_mem.used_delta, peaked_reported=cpu_mem.peaked_delta, 
            consumed_expected=128,                peaked_expected=128,  abs_tol=1)
check_match(consumed_reported=gpu_mem.used_delta, peaked_reported=gpu_mem.peaked_delta, 
            consumed_expected=1024,               peaked_expected=1024, abs_tol=1)

# cleanup
del cpu2, gpu2

'test_data_accessor'

RAM: Consumed Peaked  Used Total | Exec time 0.005s
CPU:        0      0     2159 MB |
GPU:    -1024   1024      996 MB |


## .stop

In [12]:
"""test_stop"""
exp1.cl.stop()
#check that no output appears after this one

'test_stop'

RAM: Consumed Peaked  Used Total | Exec time 0.028s
CPU:        0      0     2159 MB |
GPU:        0      0      996 MB |


In [13]:
%%capture output
cpu1 = consume_cpu_ram_128mb()

In [14]:
"""test_report"""
output = str(output)
print_output(output)
assert output == "", "there should be no output as logger has been stopped"

# cleanup
del cpu1

'test_report'

Captured output:
--------------------------------------------------
--------------------------------------------------



In [15]:
%%javascript # prevent committing an unsaved notebook
IPython.notebook.save_notebook()

<IPython.core.display.Javascript object>